In [1]:
import pandas as pd
from generator_mamdani import MamdaniRuleBaseGenerator
from rete_network import ReteNetwork
from anfis import ANFIS
import random

In [2]:
weight = {
    "x1": 0.154,
    "x2": 0.225,
    "x3": 0.354,
    "x4": 0.043,
    "x5": 0.124,
    "x6": 0.1,
}

risk_reducing = {"x4", "x5", "x6"}

gen = MamdaniRuleBaseGenerator() 
all_rules = gen.generate_rules()
random.shuffle(all_rules)
#all_rules = all_rules[:1000]

split_idx = int(0.8 * len(all_rules))
train_rules = all_rules[:split_idx]
test_rules = all_rules[split_idx:]

rete_train = ReteNetwork(train_rules)
anfis = ANFIS(rete_train)
anfis.initialize_coeffs(weight, risk_reducing)

In [3]:
anfis.train_base(lr=0.01, epochs=50, batch_size=128)

Epoch 1/50, MSE=0.014701
Epoch 2/50, MSE=0.014701
Epoch 3/50, MSE=0.014700
Epoch 4/50, MSE=0.014700
Epoch 5/50, MSE=0.014700
Epoch 6/50, MSE=0.014699
Epoch 7/50, MSE=0.014699
Epoch 8/50, MSE=0.014699
Epoch 9/50, MSE=0.014699
Epoch 10/50, MSE=0.014698
Epoch 11/50, MSE=0.014698
Epoch 12/50, MSE=0.014698
Epoch 13/50, MSE=0.014698
Epoch 14/50, MSE=0.014697
Epoch 15/50, MSE=0.014697
Epoch 16/50, MSE=0.014697
Epoch 17/50, MSE=0.014697
Epoch 18/50, MSE=0.014696
Epoch 19/50, MSE=0.014696
Epoch 20/50, MSE=0.014696
Epoch 21/50, MSE=0.014696
Epoch 22/50, MSE=0.014695
Epoch 23/50, MSE=0.014695
Epoch 24/50, MSE=0.014695
Epoch 25/50, MSE=0.014694
Epoch 26/50, MSE=0.014694
Epoch 27/50, MSE=0.014694
Epoch 28/50, MSE=0.014694
Epoch 29/50, MSE=0.014693
Epoch 30/50, MSE=0.014693
Epoch 31/50, MSE=0.014693
Epoch 32/50, MSE=0.014693
Epoch 33/50, MSE=0.014692
Epoch 34/50, MSE=0.014692
Epoch 35/50, MSE=0.014692
Epoch 36/50, MSE=0.014692
Epoch 37/50, MSE=0.014691
Epoch 38/50, MSE=0.014691
Epoch 39/50, MSE=0.01

In [4]:
anfis.save_model("anfis_base.pkl")

Model saved in anfis_base.pkl


In [5]:
rete_test = ReteNetwork(test_rules)
#anfis.load_model("anfis_base.pkl")
results = anfis.check_model_adequacy(rete_test)
print(results)

{'MAE': 0.09710207816974896, 'RMSE': 0.11959901882112187, 'Correlation': 0.9999294342564388}


In [6]:
anfis.get_global_polynomial_equation()

'R = sigmoid(-0.0005 + 0.1540*x1 + 0.2250*x2 + 0.3540*x3 + -0.0431*x4 + -0.1241*x5 + -0.1001*x6)'

In [ ]:
anfis.train_and_reduce(
    initial_epochs=30, 
    batch_size=128, 
    reduction_epochs=20, 
    significance_threshold=0.05,
    load_filepath="anfis_base.pkl"
)

Model load from anfis_base.pkl

Model reduction, round 1
R = sigmoid(-0.0005 + 0.1540*x1 + 0.2250*x2 + 0.3540*x3 + -0.1241*x5 + -0.1001*x6)
Retraining the model after reduction, epochs=20
Epoch 1/20, MSE=0.015635
Epoch 2/20, MSE=0.015634
Epoch 3/20, MSE=0.015634
Epoch 4/20, MSE=0.015634
Epoch 5/20, MSE=0.015634
Epoch 6/20, MSE=0.015633
Epoch 7/20, MSE=0.015633
Epoch 8/20, MSE=0.015633
Epoch 9/20, MSE=0.015633
Epoch 10/20, MSE=0.015632
Epoch 11/20, MSE=0.015632
Epoch 12/20, MSE=0.015632
Epoch 13/20, MSE=0.015632
Epoch 14/20, MSE=0.015631
Epoch 15/20, MSE=0.015631
Epoch 16/20, MSE=0.015631
Epoch 17/20, MSE=0.015631
Epoch 18/20, MSE=0.015630
Epoch 19/20, MSE=0.015630
Epoch 20/20, MSE=0.015630

Model reduction, round 2
R = sigmoid(-0.0005 + 0.1539*x1 + 0.2249*x2 + 0.3540*x3 + -0.1241*x5 + -0.1001*x6)
Retraining the model after reduction, epochs=20
Epoch 1/20, MSE=0.015630
Epoch 2/20, MSE=0.015629
Epoch 3/20, MSE=0.015629
Epoch 4/20, MSE=0.015629
Epoch 5/20, MSE=0.015629
Epoch 6/20, MSE=0.0

In [8]:
anfis.save_model("anfis_final.pkl")

Model saved in anfis_final.pkl


In [9]:
results = anfis.check_model_adequacy(rete_test)
print(results)
anfis.get_global_polynomial_equation()

{'MAE': 0.09995382446370775, 'RMSE': 0.12322504991487937, 'Correlation': 0.996685891660865}


'R = sigmoid(-0.0007 + 0.1539*x1 + 0.2249*x2 + 0.3540*x3 + -0.1242*x5 + -0.1002*x6)'

In [8]:
anfis = ANFIS()
anfis.load_model("anfis_final.pkl")
test_samples = [
    {"x1": 0.1, "x2": 0.15, "x3": 0.1, "x4": 0.97, "x5": 0.15, "x6": 0.15},
    {"x1": 0.1, "x2": 0.15, "x3": 0.1, "x4": 0.97, "x5": 0.15, "x6": 0.42},
    {"x1": 0.1, "x2": 0.15, "x3": 0.1, "x4": 0.97, "x5": 0.15, "x6": 0.55},
    {"x1": 0.1, "x2": 0.15, "x3": 0.1, "x4": 0.97, "x5": 0.15, "x6": 0.87},
]

for i, sample in enumerate(test_samples, start=1):
    y_pred = anfis.predict(sample)
    print(f"Sample {i}: input={sample}, predict={y_pred:.4f}")


Model load from anfis_final.pkl
Sample 1: input={'x1': 0.1, 'x2': 0.15, 'x3': 0.1, 'x4': 0.97, 'x5': 0.15, 'x6': 0.15}, predict=0.5122
Sample 2: input={'x1': 0.1, 'x2': 0.15, 'x3': 0.1, 'x4': 0.97, 'x5': 0.15, 'x6': 0.42}, predict=0.5054
Sample 3: input={'x1': 0.1, 'x2': 0.15, 'x3': 0.1, 'x4': 0.97, 'x5': 0.15, 'x6': 0.55}, predict=0.5021
Sample 4: input={'x1': 0.1, 'x2': 0.15, 'x3': 0.1, 'x4': 0.97, 'x5': 0.15, 'x6': 0.87}, predict=0.4942
